In [1]:
import pandas as pd
import numpy as np
import os, time, stat, io
from scipy.stats import gamma, norm
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use('fivethirtyeight')
import plotly.graph_objects as go

%load_ext autoreload
%autoreload 2
from covid_data_helper import *
from coronita_chart_helper import *
from coronita_model_helper import *

from matplotlib.backends.backend_pdf import PdfPages

In [7]:
df_nyny_raw = pd.read_csv('https://visitdata.org/data/NewYork_NewYork.csv', parse_dates=['date'])
df_nyny_raw.head()

,date,state,county,categoryid,categoryname,demo,visits,avgDuration,p50Duration
0,2020-02-01,New York,New York,4bf58dd8d48988d100951735,Pet Stores,Above65,348.0,37.0,NaN
1,2020-02-01,New York,New York,4bf58dd8d48988d100951735,Pet Stores,All,9326.0,52.0,15.0
2,2020-02-01,New York,New York,4bf58dd8d48988d100951735,Pet Stores,Below65,8978.0,53.0,NaN
3,2020-02-01,New York,New York,4bf58dd8d48988d101951735,Thrift / Vintage Stores,Above65,285.0,38.0,NaN
4,2020-02-01,New York,New York,4bf58dd8d48988d101951735,Thrift / Vintage Stores,All,9535.0,31.0,15.0


In [11]:
df_nyny = df_nyny_raw.copy()
df_nyny = df_nyny.set_index(['date','state','county','categoryname','demo'])
df_nyny

categoryid  \
date       state    county   categoryname            demo                                
2020-02-01 New York New York Pet Stores              Above65  4bf58dd8d48988d100951735   
                                                     All      4bf58dd8d48988d100951735   
                                                     Below65  4bf58dd8d48988d100951735   
                             Thrift / Vintage Stores Above65  4bf58dd8d48988d101951735   
                                                     All      4bf58dd8d48988d101951735   
...                                                                                ...   
2020-05-18 New York New York Sports                  All                         Group   
                                                     Below65                     Group   
                             Travel & Transport      Above65                     Group   
                                                     All                         Group   
                                                     Below65                     Group   

                                                                visits  \
date       state    county   categoryname            demo                
2020-02-01 New York New York Pet Stores              Above65     348.0   
                                                     All        9326.0   
                                                     Below65    8978.0   
                             Thrift / Vintage Stores Above65     285.0   
                                                     All        9535.0   
...                                                                ...   
2020-05-18 New York New York Sports                  All       30970.0   
                                                     Below65   29996.0   
                             Travel & Transport      Above65    2559.0   
                                                     All      122610.0   
                                                     Below65  120051.0   

                                                              avgDuration  \
date       state    county   categoryname            demo                   
2020-02-01 New York New York Pet Stores              Above65         37.0   
                                                     All             52.0   
                                                     Below65         53.0   
                             Thrift / Vintage Stores Above65         38.0   
                                                     All             31.0   
...                                                                   ...   
2020-05-18 New York New York Sports                  All             36.0   
                                                     Below65         37.0   
                             Travel & Transport      Above65         64.0   
                                                     All             86.0   
                                                     Below65         87.0   

                                                              p50Duration  
date       state    county   categoryname            demo                  
2020-02-01 New York New York Pet Stores              Above65          NaN  
                                                     All             15.0  
                                                     Below65          NaN  
                             Thrift / Vintage Stores Above65          NaN  
                                                     All             15.0  
...                                                                   ...  
2020-05-18 New York New York Sports                  All             10.0  
                                                     Below65          NaN  
                             Travel & Transport      Above65          NaN  
                                                     All             16.0  
                            

In [12]:
df_nyny.categoryid.unique()

array(['4bf58dd8d48988d100951735', '4bf58dd8d48988d101951735',
       '4bf58dd8d48988d102941735', '4bf58dd8d48988d103951735',
       '4bf58dd8d48988d104941735', '4bf58dd8d48988d104951735',
       '4bf58dd8d48988d106951735', '4bf58dd8d48988d107951735',
       '4bf58dd8d48988d108951735', '4bf58dd8d48988d10a951735',
       '4bf58dd8d48988d10c941735', '4bf58dd8d48988d10c951735',
       '4bf58dd8d48988d10e941735', '4bf58dd8d48988d10f941735',
       '4bf58dd8d48988d10f951735', '4bf58dd8d48988d110941735',
       '4bf58dd8d48988d110951735', '4bf58dd8d48988d111941735',
       '4bf58dd8d48988d111951735', '4bf58dd8d48988d112941735',
       '4bf58dd8d48988d112951735', '4bf58dd8d48988d113941735',
       '4bf58dd8d48988d113951735', '4bf58dd8d48988d114951735',
       '4bf58dd8d48988d116941735', '4bf58dd8d48988d117951735',
       '4bf58dd8d48988d118941735', '4bf58dd8d48988d118951735',
       '4bf58dd8d48988d119951735', '4bf58dd8d48988d11b941735',
       '4bf58dd8d48988d11d941735', '4bf58dd8d48988d11e9